In [1]:
from critdd import Diagram
import pandas as pd

# download example data
df = pd.read_excel("manual_set_results_test_fitness_size.xlsx")

In [2]:
variants = variants_to_show = [
    'VARIANT 1b', 
    'VARIANT 2', 
    'VARIANT 3b', 
    'VARIANT 4b', 
    'VARIANT 5', 
    'VARIANT 6b', 
    'VARIANT 7',
    'VARIANT 8'
]

In [3]:
cols = list(df.columns)

In [4]:
variant_col_idxs = [cols.index(variant) for variant in variants]

In [5]:
variant_col_idxs

[49, 13, 55, 61, 31, 67, 43, 85]

In [6]:
variant_smallest_means_cols = [cols[idx + 1] for idx in variant_col_idxs]
variant_optimal_means_cols = [cols[idx + 3] for idx in variant_col_idxs]
variant_best_means_cols = [cols[idx + 5] for idx in variant_col_idxs]

In [7]:
{df.columns[col_idx+ 1]: variants[idx] for idx, col_idx in enumerate(variant_col_idxs)}

{'Unnamed: 50': 'VARIANT 1b',
 'Unnamed: 14': 'VARIANT 2',
 'Unnamed: 56': 'VARIANT 3b',
 'Unnamed: 62': 'VARIANT 4b',
 'Unnamed: 32': 'VARIANT 5',
 'Unnamed: 68': 'VARIANT 6b',
 'Unnamed: 44': 'VARIANT 7',
 'Unnamed: 86': 'VARIANT 8'}

In [8]:
smallest_means_df = df[variant_smallest_means_cols]
smallest_means_df = smallest_means_df.rename(columns={df.columns[col_idx+ 1]: variants[idx] for idx, col_idx in enumerate(variant_col_idxs)})
optimal_means_df = df[variant_optimal_means_cols]
optimal_means_df = optimal_means_df.rename(columns={df.columns[col_idx + 3]: variants[idx] for idx, col_idx in enumerate(variant_col_idxs)})
best_means_df = df[variant_best_means_cols]
best_means_df = best_means_df.rename(columns={df.columns[col_idx + 5]: variants[idx] for idx, col_idx in enumerate(variant_col_idxs)})

In [9]:
def wrangle_individual(individual_df: pd.DataFrame):

    df_fitness = individual_df[2:16].map(lambda v: float(v.split()[0]) if type(v) != float else 0.0)
    df_size = smallest_means_df[23:].map(lambda v: float(v.split()[0]) if type(v) != float else 0.0)    

    return df_fitness, df_size


In [13]:
def export_cdd_diagram(output_file: str, df: pd.DataFrame):

    # create a CD diagram from the Pandas DataFrame
    diagram = Diagram(
        df.to_numpy(),
        treatment_names = df.columns,
        maximize_outcome = False
    )

    # inspect average ranks and groups of statistically indistinguishable treatments
    diagram.average_ranks # the average rank of each treatment
    diagram.get_groups(alpha=.05, adjustment="holm")

    # export the diagram to a file
    diagram.to_file(
        output_file,
        as_document=True,
        alpha = .05,
        adjustment = "holm",
        reverse_x = True,
        axis_options = {"title": "critdd"},
    )


In [11]:
output_mapping_dict = {
    "smallest": smallest_means_df,
    "optimal": optimal_means_df,
    "best": best_means_df
}

In [14]:
for output_file_base_name, individual_df in output_mapping_dict.items():

    fitness_df, size_df = wrangle_individual(individual_df)
    
    fitness_filename = output_file_base_name + "_fitness.tex"
    size_filename = output_file_base_name + "_size.tex"

    # fitness
    export_cdd_diagram(f"cdd_plots/{fitness_filename}", fitness_df)
    # size
    export_cdd_diagram(f"cdd_plots/{size_filename}", size_df)

ValueError: zero_method 'wilcox' and 'pratt' do not work if x - y is zero for all elements.

In [16]:
fitness_df, size_df = wrangle_individual(best_means_df)

In [17]:
fitness_df

,VARIANT 1b,VARIANT 2,VARIANT 3b,VARIANT 4b,VARIANT 5,VARIANT 6b,VARIANT 7,VARIANT 8
2,4.88760,5.37385,4.93861,5.32850,5.45337,5.32850,5.44963,4.92779
3,604.79234,448.38704,283.62398,611.18369,448.95808,611.18369,448.89562,295.14712
4,30.02267,30.43734,30.49417,29.50232,30.38595,29.50232,30.31073,30.11626
5,5.47069,6.00842,5.15218,5.56449,5.78519,5.56449,5.66997,5.00023
6,0.25830,0.24857,0.25541,0.26993,0.25509,0.26993,0.25508,0.25624
7,7.86223,7.91244,11.17357,7.91915,8.12025,7.91915,8.12365,8.10630
8,8.56155,10.23339,8.88918,9.11088,10.42694,9.11088,10.34252,9.26588
9,69.44008,58.25503,57.58622,72.69685,57.92340,72.69685,57.93185,57.72051
10,3.58509,3.52352,3.36649,3.65122,3.56183,3.65122,3.54255,3.37234
11,3.20459,3.27716,2.95510,3.41882,3.48953,3.41882,3.48069,3.11345
